In [2]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set(style='whitegrid')

from IPython.display import clear_output
from six.moves import urllib

In [3]:
import os
for dirname, _, filenames in os.walk("input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

input\ks-projects-201612.csv
input\ks-projects-201801.csv


In [4]:
prj2018 = pd.read_csv("input\ks-projects-201801.csv")

In [5]:
prj2018.pop('name')
prj2018.pop('ID')
prj2018.pop('usd pledged')

0            0.0
1          100.0
2          220.0
3            1.0
4         1283.0
           ...  
378656      25.0
378657     155.0
378658      20.0
378659     200.0
378660     524.0
Name: usd pledged, Length: 378661, dtype: float64

In [6]:
deadlines = prj2018.pop('deadline')
launches = prj2018.pop('launched')
deadlines.head()

0    2015-10-09
1    2017-11-01
2    2013-02-26
3    2012-04-16
4    2015-08-29
Name: deadline, dtype: object

In [7]:
count = 0
new_data = pd.DataFrame({'time':int()},index = deadlines.index, columns = ['time'],dtype=int) 
for x in deadlines:
    temp_str_1 = str(deadlines.iloc[count])
    temp_str_2 = str(launches.iloc[count])
    
    temp_int_1_y = int(temp_str_1[0:4])
    temp_int_2_y = int(temp_str_2[0:4])
    temp_int_1_m = int(temp_str_1[5:7])
    temp_int_2_m = int(temp_str_2[5:7])
    temp_int_1_d = int(temp_str_1[8:])
    temp_int_2_d = int(temp_str_2[8:10])
    
    temp_int = (temp_int_1_y - temp_int_2_y)*365 + (temp_int_1_m - temp_int_2_m)*30 + (temp_int_1_d - temp_int_2_d) + 1.0 - 1.0
    #new_data.loc[count] = np.float64(temp_int)
    new_data.loc[count] = temp_int
    count += 1
new_data.head()

,time
0,58.0
1,59.0
2,44.0
3,29.0
4,55.0


In [8]:
prj2018 = prj2018.join(new_data)

In [9]:
prj2018 = prj2018.query('state != "live"')

In [10]:
states = pd.DataFrame({'state':int()},index = prj2018.index, columns = ['state'],dtype=int)
states.loc[prj2018['state'] == "successful", 'state'] = 1
states.loc[prj2018['state'] != "successful", 'state'] = 0

In [11]:
prj2018.pop('state')
prj2018 = prj2018.join(states)

In [21]:
prj2018["category"] = prj2018["category"].astype('category')
prj2018["main_category"] = prj2018["main_category"].astype('category')
prj2018["currency"] = prj2018["currency"].astype('category')
prj2018["country"] = prj2018["category"].astype('category')
prj2018.dtypes

category            category
main_category       category
currency            category
goal                 float64
pledged              float64
backers                int64
country             category
usd_pledged_real     float64
usd_goal_real        float64
time                 float64
state                  int32
dtype: object

In [22]:
prj2018["category"] = prj2018["category"].cat.codes
prj2018["main_category"] = prj2018["main_category"].cat.codes
prj2018["currency"] = prj2018["currency"].cat.codes
prj2018["country"] = prj2018["country"].cat.codes
prj2018.head()

,category,main_category,currency,goal,pledged,backers,country,usd_pledged_real,usd_goal_real,time,state
0,108,12,5,1000.0,0.0,0,108,0.0,1533.95,58.0,0
1,93,6,13,30000.0,2421.0,15,93,2421.0,30000.00,59.0,0
2,93,6,13,45000.0,220.0,3,93,220.0,45000.00,44.0,0
3,90,10,13,5000.0,1.0,1,90,1.0,5000.00,29.0,0
4,55,6,13,19500.0,1283.0,14,55,1283.0,19500.00,55.0,0


In [26]:
x_prj2018_trn = prj2018.iloc[:340795,:]
x_prj2018_eval = prj2018.iloc[340795:,:]

In [27]:
y_prj2018_trn = x_prj2018_trn.pop('state')
y_prj2018_eval = x_prj2018_eval.pop('state')

In [28]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu'),  
    keras.layers.Dense(10, activation='softmax')
])

In [29]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [30]:
model.fit(x_prj2018_trn, y_prj2018_trn, epochs=10)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

10650/10650 [==============================] - 9s 886us/step - loss: 22.0827 - accuracy: 0.9787
Epoch 2/10
10650/10650 [==============================] - 9s 883us/step - loss: 12.6758 - accuracy: 0.9832
Epoch 3/10
10650/10650 [==============================] - 10s 916us/step - loss: 6.1800 - accuracy: 0.9853
Epoch 4/10
10650/10650 [==============================] - 9s 855us/step - loss: 2.6999 - accuracy: 0.9865
Epoch 5/10
10650/10650 [==============================] - 10s 920us/step - loss: 

In [31]:
test_loss, test_acc = model.evaluate(x_prj2018_eval,  y_prj2018_eval, verbose=1) 

print('Test accuracy:', test_acc)

1096/1096 [==============================] - 1s 763us/step - loss: 0.0369 - accuracy: 0.9923
Test accuracy: 0.9922719597816467
